# **PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS - NÚCLEO DE EDUCAÇÃO A DISTÂNCIA**

### Pós-graduação Lato Sensu em Ciência de Dados e Big Data


### **2. Processamento/Tratamento dos dados**

In [ ]:
## Importação das Bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

**Carregando a base**


In [ ]:
df_rais2019CE = pd.read_csv('./BD_Rais_2019_CE.csv')
df_rais2019CE.head()

**Verificando a quantidade de linhas/atributos e tipos atribuídos**

In [ ]:
print('Dimensões da Base de Dados: {} linhas e {} colunas'.format(len(df_rais2019CE), len(df_rais2019CE.columns)),'\n\n')
df_rais2019CE.info()

In [ ]:
# Removendo Colunas desnecessárias
df_rais2019CE = df_rais2019CE.drop(columns=['ano','sigla_uf','id_municipio_6','natureza_estabelecimento','tipo_estabelecimento',
                                            'cnae_1','subatividade_ibge','bairros_sp','distritos_sp','bairros_fortaleza',
                                            'bairros_rj','regioes_administrativas_df'])
print('Dimensões da Base de Dados: {} linhas e {} colunas'.format(len(df_rais2019CE), len(df_rais2019CE.columns)),'\n\n')
df_rais2019CE.info()

In [ ]:
# Ajustando Tipo da Coluna, conforme base original
tipo_coluna = { "id_municipio": "string",
                "natureza_juridica": "string",
                "cnae_2": "string",
                "cnae_2_subclasse": "string",
                "subsetor_ibge": "string",
                "cep_estabelecimento": "string"}
df_rais2019CE = df_rais2019CE.astype(tipo_coluna)

## as colunas "cnae_2", "cnae_2_subclasse" e "subsetor_ibge" precisam ser corrigidas colocando um "0" a esquerda nos respectivos códigos 

df_rais2019CE['cnae_2'] = df_rais2019CE['cnae_2'].str.rstrip('0')                 # Esta coluna era float64, por isso tem que tirar o ".0" 
df_rais2019CE['cnae_2'] = df_rais2019CE['cnae_2'].str.rstrip('.').str.zfill(5)    # do final antes de preencher com zero a esquerda
                                                                                  

df_rais2019CE['cnae_2_subclasse'] = df_rais2019CE['cnae_2_subclasse'].str.zfill(7) 
df_rais2019CE['subsetor_ibge'] = df_rais2019CE['subsetor_ibge'].str.zfill(2) 
df_rais2019CE.dtypes

In [ ]:
## Carregando os demais datasets
df_cnae = pd.read_csv('./BD_CNAE2.csv')
df_municipios = pd.read_csv('./BD_Municipios_CE.csv')
df_natjur = pd.read_csv('./BD_Nat_Juridica.csv')

In [ ]:
## Definindo função para verificar integridade de colunas

def verifica_integridade(nome_coluna, nome_tabela, nome_coluna_tb):
 
  lista = df_rais2019CE[nome_coluna].unique()
  coluna_ok = True
  
  for codigo in lista:
    p_lista = []
    p_lista.append(codigo)
    x = nome_tabela.loc[nome_tabela[nome_coluna_tb].isin(p_lista)]
    if x.empty:
      coluna_ok = False
      print('Coluna: "{}" - Código {} não encontrado'.format(nome_coluna,codigo))
  if coluna_ok:
    print('Coluna: "{}" - todos valores válidos'.format(nome_coluna))

In [ ]:
verifica_integridade('id_municipio', df_municipios, 'id_municipio')
verifica_integridade('natureza_juridica', df_natjur, 'cod_nat_juridica')
verifica_integridade('cnae_2', df_cnae, 'classe_id')
verifica_integridade('cnae_2_subclasse', df_cnae, 'subclasse_id')

In [ ]:
print(df_rais2019CE.loc[df_rais2019CE['cnae_2'].isnull()])
print(df_rais2019CE.loc[df_rais2019CE['cnae_2_subclasse'] == '9999999'])

In [ ]:
print(df_rais2019CE.iloc[13001])  ## Visualizando o registro

In [ ]:
df_rais2019CE.drop(index=13001, inplace=True)   ## Excluindo o registro do dataset

In [ ]:
## Verificação da coluna "subsetor_ibge"
## Os valores de "subsetor_ibge" podem assumir os valores 01 a 25
subsetor_ibge_cod = list(range(1,26))
subsetor_ibge_cod = list(map(str, subsetor_ibge_cod))
subsetor_ibge_cod = [cod.zfill(2) for cod in subsetor_ibge_cod]

# As respectivas descrições
subsetor_ibge_desc = ['Extrativa mineral','Indústria de produtos minerais nao metálicos','Indústria metalúrgica','Indústria mecânica','Indústria do material elétrico e de comunicaçoes',
                      'Indústria do material de transporte','Indústria da madeira e do mobiliário','Indústria do papel, papelao, editorial e gráfica',
                      'Ind. da borracha, fumo, couros, peles, similares, ind. diversas','Ind. química de produtos farmacêuticos, veterinários, perfumaria',
                      'Indústria têxtil do vestuário e artefatos de tecidos','Indústria de calçados','Indústria de produtos alimentícios, bebidas e álcool etílico',
                      'Serviços industriais de utilidade pública','Construçao civil','Comércio varejista','Comércio atacadista','Instituiçoes de crédito, seguros e capitalizaçao',
                      'Com. e administraçao de imóveis, valores mobiliários, serv. Técnico','Transportes e comunicaçoes','Serv. de alojamento, alimentaçao, reparaçao, manutençao, redaçao',
                      'Serviços médicos, odontológicos e veterinários','Ensino','Administraçao pública direta e autárquica','Agricultura, silvicultura, criaçao de animais, extrativismo vegetal']

subsetor_ibge_dict = {'subsetor_cod': subsetor_ibge_cod, 'subsetor_descricao': subsetor_ibge_desc}

## Vamos criar um Dataset Pandas para usar a mesma função "vrifica_integridade"
df_subsetor_ibge = pd.DataFrame.from_dict(subsetor_ibge_dict) 

verifica_integridade('subsetor_ibge', df_subsetor_ibge, 'subsetor_cod')     # Verificando a coluna

In [ ]:
## Verificação da coluna "tamanho_estabelecimento" (valores de 1 a 10
print(df_rais2019CE.loc[df_rais2019CE['tamanho_estabelecimento'] < 1].empty)   # Testa se não tem nenhum menor que 1
print(df_rais2019CE.loc[df_rais2019CE['tamanho_estabelecimento'] > 10].empty)  # Testa se não tem nenhum maior que 10

In [ ]:
df_rais2019CE.describe()

In [ ]:
# Verificando Dados ausentes (missing data)
df_rais2019CE.isnull().sum()

In [ ]:
## Salvando o Dataset RAIS após tratamento
df_rais2019CE.to_csv('BD_Rais_2019_CE_02.csv', index=False)